## RAVDESS数据库的读取

In [36]:
#导入响应的包和文件路径
import os
import pandas as pd
import numpy as np
import librosa
from scipy import signal
file= r'C:\Users\Administrator\Desktop\RAVDESS'

In [4]:
#查看路径下所有的文件夹
actor_folder=os.listdir(file)
print(actor_folder)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [9]:
#用pandas来包括数据
columns=["path","source","actor","gender","intensity","statement","repetition","emotion"]
data_df= pd.DataFrame(columns=columns)
count=0
for i in actor_folder:
    wav_list= os.listdir(os.path.join(file,i))
    for f in wav_list:
        wav_name= f.split(".")[0].split("-")
        wav_path= os.path.join(os.path.join(file,i),f)
        src= int(wav_name[1])
        actor= int(wav_name[-1])
        emotion= int(wav_name[2])
        if int(actor)%2==0:
            gender= "female"
        else:
            gender= "male"
        if wav_name[3]=="01":
            intensity= 0
        else:
            intensity=1
        if wav_name[4]=="01":
            statement="kids are talking by the door"
        else:
            statement="Dogs are sitting by the door"
        if wav_name[5]=="01":
            repeat=0
        else:
            repeat=1
        data_df.loc[count]=[wav_path,src,actor,gender,intensity,statement,repeat,emotion]
        count+=1

In [16]:
print("一共有%d条语音"%len(data_df))
data_df.head(5)

一共有1440条语音


,path,source,actor,gender,intensity,statement,repetition,emotion
0,C:\Users\Administrator\Desktop\RAVDESS\Actor_0...,1.0,1.0,male,0.0,kids are talking by the door,0.0,1.0
1,C:\Users\Administrator\Desktop\RAVDESS\Actor_0...,1.0,1.0,male,0.0,kids are talking by the door,1.0,1.0
2,C:\Users\Administrator\Desktop\RAVDESS\Actor_0...,1.0,1.0,male,0.0,Dogs are sitting by the door,0.0,1.0
3,C:\Users\Administrator\Desktop\RAVDESS\Actor_0...,1.0,1.0,male,0.0,Dogs are sitting by the door,1.0,1.0
4,C:\Users\Administrator\Desktop\RAVDESS\Actor_0...,1.0,1.0,male,0.0,kids are talking by the door,0.0,2.0


In [39]:
#绘制语音的波形图和频域谱
example= data_df.path[1021]
print(example)
sample, sample_rate= librosa.load(example)
print("样本有%d长"%len(sample))
print("样本采样率为%d"%sample_rate)

C:\Users\Administrator\Desktop\RAVDESS\Actor_18\03-01-01-01-01-02-18.wav
样本有77989长
样本采样率为22050


In [45]:
#绘制频谱图，使用scipy.signal.spectrogram
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

scipy.signal.spectrogram[参数说明](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html)  
scipy.signal.spectrogram(x, fs=1.0, window=("tukey",0.25),nperseg=None, noverlap=None, nfft=None,detrend="constant",scaling="density",axis=-1,mode="psd)  
x：时域信号  
fs：采样频率    
window：窗的种类，通常选择汉明窗或者汉宁窗  
nperseg：帧长  
noverlap：帧移  
nfft：使用的FFT点数  

In [49]:
freqs, times, spectrogram = log_specgram(sample, sample_rate)